In [ ]:
# 從 TensorFlow CNN tutorials 改成可以參加 kaggle

In [ ]:
# 嘗試紀錄
# 1. 有 / 255 結果比較好
# 2. 為了增加training data 所以減少val_split 結果比較好
# 3. 加上 error_enhance 好一點點

# Score: 0.99267
# 很多前面排名的 有自己增加 dataset (加上 tensorflow 的 mnist data set)

In [ ]:
import tensorflow as tf

from tensorflow.keras import datasets, layers, models
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd

import os

In [ ]:
# settings
test_data_path = './input/test.csv'
train_data_path = './input/train.csv'
error_enhance_path = ['./input/error_enhance', '.csv']

checkpoint_path = "./checkpoint/train_checkpoint"
checkpoint_dir = "./checkpoint"
final_checkpoint = './checkpoint/final_checkpoint'
final_model = './saved_model/final_model'

In [ ]:
# training settings

class CNN_set:
    seed = 200 #67
    using_enhance_data = True # 要在 training data 裡面加強 error data 嗎?
    output_enhace = False # 要輸出 enhance error 資料嗎?
    val_split = 0.001 # 要從 training data 裡面撥多少資料到 validation_data
    train_from_begin = True  # False # True
    save_checkPoint = False
    lr = [(13, 0.001), (7, 0.0003), (4, 0.0001)] # learning_rate [(每次epoch次數, 使用的 learing rate)]
    # 我看線圖 感覺原本lr就會慢慢下降 (因為每次換 lr 準確率會突然下降)
        # 查資料看起來是固定的
            # 應該是 momentum_optimizer_value 的問題 但目前不知道在哪裡設定
        # 有一個參數可以設定 : decay 預設是0
#     decay = 0.05
    
#     img_size = (28, 28)

# load and resize data 

In [ ]:
# 讀取資料
train_val_data = pd.read_csv(train_data_path)
# 取得 train_data 的 label
train_val_label = train_val_data['label']
# 取得 train_data 的 image
train_val_pic = train_val_data.iloc[:,1:].values.reshape(-1,28,28,1)  # 從1開始是因為第一行是 label 

print('train_val_label shape:', train_val_label.shape)
print('train_val_pic shape:', train_val_pic.shape)

In [ ]:
# add error enhance data (to balance the data, because I think odd written data isnt enough)
have_enhance_data = False
if CNN_set.using_enhance_data :
    file_count = 1
    while True :
        each_error_path = error_enhance_path[0]+str(file_count)+error_enhance_path[1]
    #     print(each_error_path)
        if os.path.isfile(each_error_path) : 
            have_enhance_data = True
            err_enh_data = pd.read_csv(each_error_path)
            err_enh_label = err_enh_data['label']
            # 取得 train_data 的 image
            err_enh_pic = err_enh_data.iloc[:,1:].values.reshape(-1,28,28,1)  # 從1開始是因為第一行是 label 
            # 取得 test_data 的 image

            print('err_enh_label shape:', err_enh_label.shape)
            print('err_enh_pic shape:', err_enh_pic.shape)

            for _ in range(3):  # 看要複製多少份
                # err_enh 合併到 train_val
                train_val_label = np.concatenate([train_val_label,err_enh_label],axis=0)
                # 如果是 pd
                # train_val_pic = train_val_pic.merge(err_enh_pic, how='outer', left_index=True, right_index=True)
                train_val_pic = np.concatenate([train_val_pic,err_enh_pic],axis=0)
        else :
            break
        file_count += 1

print('train_val_label shape:', train_val_label.shape)
print('train_val_pic shape:', train_val_pic.shape)

In [ ]:
# 確認 err_enh_pic 沒問題  只會印最後一個enhance檔案裏面的圖案
if have_enhance_data :
    class_names = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
    plt.figure(figsize=(7,2))
    for i in range(min(14, len(err_enh_pic))):
        plt.subplot(2,7,i+1)
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        plt.imshow(err_enh_pic[i], cmap=plt.get_cmap('gray'))
        plt.xlabel(class_names[err_enh_label[i]])
    plt.show()

In [ ]:
# 確認 train_val_pic 沒問題
class_names = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
plt.figure(figsize=(7,2))  # 設定這個 plt 要顯示的大小
for i in range(14):
    plt.subplot(2,7,i+1)     # 切換到對應要顯示的位置
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_val_pic[i], cmap=plt.get_cmap('gray'))  # 這裡要注意不管有沒有 /255，印出來的圖片都會長的一樣
    plt.xlabel(class_names[int(train_val_label[i])])
plt.show()

In [ ]:
# 確定資料沒問題後 開始調整至符合 input 跟 output 的形狀
train_val_pic = train_val_pic.reshape((train_val_label.shape[0], 28, 28, 1)).astype('float64') /255.0
# train_val_pic = train_val_pic.reshape((42000, 28, 28, 1)).astype('float64') /255

# 這裡不需要改變 lable 形狀
# train_val_lable = to_categorical(train_val_lable, num_classes=class種類)

print('train_val_pic shape:', train_val_pic.shape)

In [ ]:
# 通常會用 test_data 當作 valid_data
# 但因為kaggle沒有給 所以就再從train_data分出來

# Split data
from sklearn.model_selection import train_test_split
train_pic, val_pic , train_label, val_label = train_test_split(train_val_pic, train_val_label, 
    test_size=CNN_set.val_split, random_state=CNN_set.seed)
    # random_state 如果想要重現一樣的結果 就代入同樣的數字 有點像 random seed

# 這時候就不能再用 for 印 plot 
# 因為 dataset 已經分成兩組 且打亂了
# 所以有些 位置 已經被抽走了 所以會造成 Key error
    
print('train_pic shape:', train_pic.shape)
print('val_pic shape:', val_pic.shape)
print('train_label shape:', train_label.shape)
print('val_label shape:', val_label.shape)

In [ ]:
# 作弊使用 tensorflow 的 mnist dataset
# of course geting Score: 1.00000 # because model already knows the answer
# conclusion : CNN model can handle this question, but cannot deal with unseen data 

# from tensorflow.python.keras.datasets import mnist
# (train_pic, train_label), (val_pic, val_label) = mnist.load_data() # 讀進來是還沒 /255 的

# train_pic = train_pic /255
# val_pic = val_pic /255

# train_pic = train_pic.reshape((train_pic.shape[0], 28, 28, 1))
# val_pic = val_pic.reshape((val_pic.shape[0], 28, 28, 1))

# # val_pic 通常不能和 train_pic 有交集
# # 但這裡為了最大化 training data 所以加在一起
# train_pic = np.concatenate([train_pic,val_pic],axis=0)
# train_label = np.concatenate([train_label,val_label],axis=0)

# print('train_pic shape:', train_pic.shape)
# print('val_pic shape:', val_pic.shape)
# print('train_label shape:', train_label.shape)
# print(train_label)
# print('val_label shape:', val_label.shape)

# create the model

In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10))

In [ ]:
model.summary()

# loading checkpoint

In [ ]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
if CNN_set.train_from_begin==False and latest != None :
    print("using check points :", latest)
    model.load_weights(latest)
else :
    print("training from begin")
    

In [ ]:
# save training process history
his_acc = []
his_val_acc = []

# training

In [ ]:
cp_callback_list = []

# 建立 儲存 checkpoint 的 callback function
# Create a callback that saves the model's weights 
save_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, save_weights_only=True, verbose=0)
        # verbose 在 jupyter 好像沒什麼用
        # verbose = 0 不輸出, 1 顯示進度條, 2 為每一個 epoch 輸出一條紀錄

if CNN_set.save_checkPoint == False :
    print("dont save checkpoint")
    cp_callback = None
else :
    cp_callback_list.append(save_callback)

In [ ]:
# start training
for epoch, lr in CNN_set.lr:
    model.compile(optimizer = tf.keras.optimizers.Adam(lr),
          loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
          metrics=['accuracy'])

    history = model.fit(train_pic, train_label, epochs=epoch, 
                        validation_data=(val_pic, val_label),
                       callbacks=cp_callback_list)
        # callbacks 如果有給的話 每幾分鐘會觸發一次
        
    his_acc += history.history['acc']
    his_val_acc += history.history['val_acc']

In [ ]:
# 確認訓練完之後 再存一版 final
if CNN_set.save_checkPoint :
    model.save_weights(final_checkpoint)
    model.save(final_model)
    
# model.save_weights(final_checkpoint)
# model.save(final_model)

# Evaluate the model

In [ ]:
plt.plot(his_acc, label='accuracy')
plt.plot(his_val_acc, label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
# plt.ylim([0.5, 1])  # 如果要顯示數據區間
plt.legend(loc='lower right')

In [ ]:
# check final accuracy
test_loss, test_acc = model.evaluate(val_pic,  val_label, verbose=2)
print(test_loss, test_acc)

In [ ]:
# 找出 val_pic 辨識錯誤的圖片並印出來
diff_pic = []
val_y_pred = model.predict(val_pic)
val_y_pred = np.argmax(val_y_pred, axis=1)
# print(y_pred)
for pred_ans, real_ans, pic in zip(val_y_pred, val_label, val_pic):
#     print(pred_ans, real_ans, pic)
    if pred_ans != real_ans :
        diff_pic.append((pred_ans, real_ans, pic))  # 因為讀進來後有先/255
        
# 印出
rows = (len(diff_pic)//7)+1
plt.figure(figsize=(7,rows+1))  # 設定這個 plt 要顯示的大小
for i, (pred_ans, real_ans, pic) in enumerate(diff_pic):
    plt.subplot(rows,7,i+1)     # 切換到對應要顯示的位置
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(pic, cmap=plt.get_cmap('gray'))
    plt.xlabel(str(real_ans)+"->"+str(pred_ans))
plt.show()

In [ ]:
# 提升這些錯誤資料的權重 (就是提升這些辨識錯誤的資料量)
if CNN_set.output_enhace :
    error_df = pd.DataFrame()
    error_df['label'] = [x[1] for x in diff_pic]
    error_pic = [x[2].reshape(784) for x in diff_pic]
    error_pic = error_pic * 255
    for i in range(784):
        error_df['pixel'+str(i)] = [ error_pic[x][i] for x in range(len(diff_pic))]
    error_df.to_csv('error_enhance.csv', index = False)

# identify 

In [ ]:
# 取得 test_pic 的 image
test_data = pd.read_csv(test_data_path)
test_pic = test_data.iloc[:,:].values.reshape(-1,28,28,1)
print('test_pic shape:', test_pic.shape)

In [ ]:
# 確認 test_pic 沒問題
# test_pic 沒有 label
# class_names = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
plt.figure(figsize=(7,2))
for i in range(14):
    plt.subplot(2,7,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(test_pic[i], cmap=plt.get_cmap('gray'))
#     plt.xlabel(class_names[test_label[i]])
plt.show()

In [ ]:
# 特別印出某個 test case
# plt.imshow(test_pic[59])

In [ ]:
# 確認沒問題後再塑形成可以丟入 input 的形狀
test_pic = test_pic.reshape((test_pic.shape[0], 28, 28, 1)).astype('float64') /255.0 # 要注意!! 要不要 /255
print('test_pic shape:', test_pic.shape)

In [ ]:
# 辨識結果
y_pred = model.predict(test_pic)
y_pred = np.argmax(y_pred, axis=1)
print(y_pred)

In [ ]:
# 把結果輸出到 csv
submission_df = pd.DataFrame()
image_id = [i for i in range(1, 28001)]
submission_df['ImageId'] = image_id
submission_df['Label'] = y_pred
# print(submission_df)
submission_df.to_csv('submission.csv', index = False)